![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_CamemBertForTokenClassification.ipynb)

## Import ONNX CamemBertForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `CamemBertForTokenClassification` is only available since in `Spark NLP 5.2.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for token classification via `CamembertForTokenClassification` or `TFCamembertForTokenClassification`. These models are usually under `Token Classification` category and have `camembert` in their labels
- Reference: [TFCamembertForTokenClassification](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForTokenClassification)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases
- CamembertTokenizer requires the `SentencePiece` library, so we install that as well

In [3]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum sentencepiece tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.5/396.5 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [Jean-Baptiste/camembert-ner](https://huggingface.co/Jean-Baptiste/camembert-ner) model from HuggingFace as an example
- In addition to `TFCamembertForTokenClassification` we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [5]:
from optimum.onnxruntime import ORTModelForTokenClassification
import tensorflow as tf

MODEL_NAME = 'Jean-Baptiste/camembert-ner'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

(…)e/camembert-ner/resolve/main/config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

Framework not specified. Using pt to export to ONNX.


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

(…)t-ner/resolve/main/tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

(…)ner/resolve/main/special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {ONNX_MODEL}

total 433272
-rw-r--r-- 1 root root       936 Nov  3 19:15 config.json
-rw-r--r-- 1 root root 440422178 Nov  3 19:15 model.onnx
-rw-r--r-- 1 root root    810912 Nov  3 19:15 sentencepiece.bpe.model
-rw-r--r-- 1 root root       241 Nov  3 19:15 special_tokens_map.json
-rw-r--r-- 1 root root       516 Nov  3 19:15 tokenizer_config.json
-rw-r--r-- 1 root root   2418877 Nov  3 19:15 tokenizer.json


In [7]:
!mkdir {ONNX_MODEL}/assets

- As you can see, we need to move `spiece.model` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [8]:
# get label2id dictionary
labels = ort_model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(ONNX_MODEL +'/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [9]:
!mv {ONNX_MODEL}/sentencepiece.bpe.model {ONNX_MODEL}/assets

Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

In [10]:
!ls -lR {ONNX_MODEL}

onnx_models/Jean-Baptiste/camembert-ner:
total 432484
drwxr-xr-x 2 root root      4096 Nov  3 19:16 assets
-rw-r--r-- 1 root root       936 Nov  3 19:15 config.json
-rw-r--r-- 1 root root 440422178 Nov  3 19:15 model.onnx
-rw-r--r-- 1 root root       241 Nov  3 19:15 special_tokens_map.json
-rw-r--r-- 1 root root       516 Nov  3 19:15 tokenizer_config.json
-rw-r--r-- 1 root root   2418877 Nov  3 19:15 tokenizer.json

onnx_models/Jean-Baptiste/camembert-ner/assets:
total 796
-rw-r--r-- 1 root root     26 Nov  3 19:16 labels.txt
-rw-r--r-- 1 root root 810912 Nov  3 19:15 sentencepiece.bpe.model


## Import and Save CamemBertForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [11]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.1.4
setup Colab for PySpark 3.2.3 and Spark NLP 5.1.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 24.7 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [12]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `CamemBertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [13]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = CamemBertForTokenClassification.loadSavedModel(
     f"{ONNX_MODEL}",
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [14]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [15]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your CamemBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [16]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 430972
-rw-r--r-- 1 root root 440489555 Nov  3 19:18 camembert_classification_onnx
-rw-r--r-- 1 root root    810912 Nov  3 19:18 camembert_spp
drwxr-xr-x 3 root root      4096 Nov  3 19:18 fields
drwxr-xr-x 2 root root      4096 Nov  3 19:18 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForTokenClassification model 😊

In [17]:
tokenClassifier_loaded = CamemBertForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [18]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'O']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [19]:
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["Je m'appelle jean-baptiste et je vis à montréal"], ['george washington est allé à washington']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|Je m'appelle jean...|[O, O, I-PER, O, ...|
|george washington...|[I-PER, I-PER, O,...|
+--------------------+--------------------+



That's it! You can now go wild and use hundreds of `CamemBertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
